In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
precios = pd.read_csv('precios-en-surtidor-resolucin-3142016.csv')

# ¿Qué tenemos en éste dataset?

La propiedad `.shape` nos da las dimensiones que tiene el dataset (filas, columnas)

In [ ]:
precios.shape

El método `head()` nos permite ver las primeras N filas (5 por default).

In [ ]:
precios.head()

El método `describe()` nos muestra un poco de estadística descriptiba sobre aquellas columnas numéricas

In [ ]:
precios.describe()

El método `info()` nos da información sobre las columnas que tiene el dataset, la cantidad filas que tienen datos (non-null) en la misma y el tipo de dato.

In [ ]:
precios.info()

# Preparación y tranformación de datos

Cuando trabajamos con datos de fuentes externas a las cuales no tenemos control, siempre necesitaremos manipular y traformar esos datos según nuestras necesidades.

### Primer paso: Quitar horas y minutos
Lo primero que haremos, es quitar la hora y los minutos porque no nos interesan.

`.str` es una propiedad que nos permite acceder a métodos y atributos específios para la manipulación de strings.

In [ ]:
precios['fecha_vigencia'] = precios['fecha_vigencia'].str.slice(0, 10)  # '27/03/2017 10:38' -> '27/03/2017'
precios.head()

### Segundo paso: Convertir string a Date
Pandas interpretó la columna `fecha_vigencia` como string porque no conoce el formato de fecha que tiene, sin embargo, nosotros necesitamos que pandas entienda que dicha columna es una fecha. 

`.to_datetime()` convierte un string a un objeto de tipo DateTime.

In [ ]:
precios["fecha_vigencia"] = pd.to_datetime(precios["fecha_vigencia"])
precios.info()

# Algunas preguntas a responder...

### ¿Cuál es la fecha de la primer medición? ¿Y cuál fue su precio?
`.idxmin()` devuelve el índice correspondiente al menor valor de una columna determinada, en nuestro caso `fecha_vigencia`

`.loc` nos permite acceder a filas através del índice

In [ ]:
indice = precios['fecha_vigencia'].idxmin()
print(indice)

precios.loc[ indice ]

### ¿Cuál es la fecha de la última medición? ¿Y cuál fue su precio?
`.idxmax()` devuelve el índice correspondiente al mayor valor de una columna determinada, en nuestro caso `fecha_vigencia`


In [ ]:
indice = precios['fecha_vigencia'].idxmax()
print(indice)

precios.loc[ precios['fecha_vigencia'].idxmax() ]

### ¿Donde el precio de la nafta es más caro?

In [ ]:
precios.loc[ precios['precio'].idxmax() ]

### ¿Cuales son los combustibles que disponemos en el dataset?
`.unique()` nos devuelve un array con todos los valores que hay en una columna sin repetir.

In [ ]:
combustibles = precios['producto'].unique()
combustibles

### Para cada tipo de combustible... ¿Donde se encuentra el más caro en lo que va de éste mes?¿Y los más baratos?
#### los más caros
`.dt` (abreviación de **d**ate**t**ime) es una propiedad que nos permite acceder a métodos y atributos específios para la manipulación de fechas. 

`.groupby()` agrupa un conjunto de filas según una o más columnas. Es equivalente a la cláusula `GROUP BY` de `SQL`

In [ ]:
precios_mes_actual = precios[
    (precios['fecha_vigencia'].dt.month == 5) & (precios['fecha_vigencia'].dt.year == 2018) 
]

In [ ]:
precios_actuales_altos = precios_mes_actual.loc[precios_mes_actual.groupby(by='producto')['precio'].idxmax() ]
precios_actuales_altos[['fecha_vigencia', 'producto', 'empresabandera', 'provincia', 'localidad', 'precio']]

#### Los más baratos

In [ ]:
precios_actuales_altos = precios_mes_actual.loc[precios_mes_actual.groupby(by='producto')['precio'].idxmin() ]
precios_actuales_altos[['fecha_vigencia', 'producto', 'empresabandera', 'provincia', 'localidad', 'precio']]

### ¿Cuántas estaciones de servios de cada empresa hay?
`.value_counts()` devulve la cantidad de valores iguales encontrados en una columna

In [ ]:
cantidad = precios['empresabandera'].value_counts()
cantidad

`.plot()` es un método que permite realizar distintos tipos de gráficos de manera rápida.

In [ ]:
cantidad.plot(kind='bar', figsize=(15, 5))

### ¿Cómo fue el comportamiento de cada combustible en Corrientes?

In [ ]:
figure = plt.axes()
figure.figure.set_size_inches(15, 10)

for combustible in combustibles:
    combustible_corrientes = precios[
        (precios['producto'] == combustible) 
        & (precios['provincia'] == 'CORRIENTES')
    ]

    if not combustible_corrientes.empty:
        combustible_corrientes = combustible_corrientes[['fecha_vigencia', 'empresabandera', 'producto', 'precio']]
        combustible_corrientes.set_index(['fecha_vigencia'], inplace=True)
        combustible_corrientes.rename(columns={'precio': combustible}, inplace=True)

        combustible_corrientes.plot(ax=figure)
    
plt.show()

### ¿Cómo ha variado cada empresa los precios de la Nafta Super en Buenos Aires?

In [ ]:
empresas = [
    'AXION', 
    'YPF', 
    'ESSO PETROLERA ARGENTINA S.R.L', 
    'PETROBRAS', 
    'SHELL C.A.P.S.A.', 
    'OIL COMBUSTIBLES S.A.'
]

precios_baires = precios[
    (precios['empresabandera'].isin(empresas))
    & (precios['provincia'] == 'BUENOS AIRES')
    & (precios['producto'] == 'Nafta (súper) entre 92 y 95 Ron')
]

precios_baires = precios_baires[['empresabandera', 'fecha_vigencia', 'precio']]
precio_minimos_empresa_idx = precios_baires.groupby(by='empresabandera')['fecha_vigencia'].idxmin()
precio_maximos_empresa_idx = precios_baires.groupby(by='empresabandera')['fecha_vigencia'].idxmax()

In [ ]:
precio_minimos_empresa = precios_baires.loc[precio_minimos_empresa_idx].set_index('empresabandera')
precio_minimos_empresa

In [ ]:
precio_maximos_empresa = precios_baires.loc[precio_maximos_empresa_idx].set_index('empresabandera')
precio_maximos_empresa

`.join()` une dos o más DataFrame através de su índice. El parámetro `lsuffix` es una string que se agregará al final del nombre de cada columna del dataframe que se encuentra a la izquierda (En nuestro caso, precio_maximos_empresa) y `rsuffix` es lo mismo pero para el dataframe que se encuentra a la derecha (precio_minimos_empresa).

In [ ]:
precios_min_max = precio_maximos_empresa.join(precio_minimos_empresa, lsuffix='_max', rsuffix='_min')
precios_min_max

In [ ]:
precios_min_max[['precio_max', 'precio_min']].plot(kind='bar', figsize=(15, 5))

# Conclusión
El análisis de datos consiste en conocer qué datos tenemos, cuales son sus características, explorar esos datos, comprender como se comportan las variables, pero lo más importante es **Hacerse preguntas y tratar de responderlas através de los datos.**
- Fuente de datos: http://datos.gob.ar/dataset/precios-surtidor---resolucion-3142016

### Algunos tips
* Soporta a múltiples formatos para leer y escribir:
    - CSV
    - HTML
    - JSON
    - Excel
    - SQL pd.read_sql(<span style="color:#C30019"> 'SELECT * FROM some_table'</span>, connection) /  pd.to_sql(<span style="color:#C30019">'some_table'</span>, connection)
    - [Y muchos más](https://pandas.pydata.org/pandas-docs/stable/io.html)
* El límite de un `DataFrame` es el límite de la memoria RAM.
* Me gustó! ¿Por donde empiezo?
    - [Cookbook](https://pandas.pydata.org/pandas-docs/stable/10min.html)
    - [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)

# Muchas gracias!
**Medium** https://medium.com/@cristhianboujon

**LinkedIn** https://www.linkedin.com/in/cristhian-boujon/

Éste código en **Github**: https://github.com/Overflow012/talks/blob/master/pandas/an%C3%A1lisis%20surtidores.ipynb